## 보건자료 활용

In [505]:
import pandas as pd
import numpy as np
import os

## 대상자 선정

In [509]:
df_2002 = pd.read_csv('patient_2002.csv',index_col = 0)
df_2003 = pd.read_csv('patient_2003.csv',index_col = 0)

In [510]:
exclude_2002 = set(df_2002[(df_2002.MAIN_SICK.str[:3].isin(['E10','E11','E12','E13','E14'])) | (df_2002.SUB_SICK.str[:3].isin(['E10','E11','E12','E13','E14'])) | (df_2002.MAIN_SICK.str[:1].isin(['C']))].PERSON_ID)
exclude_2003 = set(df_2003[(df_2003.MAIN_SICK.str[:3].isin(['E10','E11','E12','E13','E14'])) | (df_2003.SUB_SICK.str[:3].isin(['E10','E11','E12','E13','E14'])) | (df_2003.MAIN_SICK.str[:1].isin(['C']))].PERSON_ID)


In [514]:
exclude_2002_2003 = exclude_2002 | exclude_2003

In [515]:
len(exclude_2002_2003)

54726

In [608]:
df_2004 = pd.read_csv('patient_2004.csv',index_col = 0)
df_2005 = pd.read_csv('patient_2005.csv',index_col = 0)

In [610]:
df_2004_2005 = pd.concat([df_2004,df_2005])

In [611]:
df_2004_2005 = df_2004_2005[(df_2004_2005.MAIN_SICK.str[:3].isin(['E10','E11','E12','E13','E14'])) | (df_2004_2005.SUB_SICK.str[:3].isin(['E10','E11','E12','E13','E14'])) | (df_2004_2005.MAIN_SICK.str[:1].isin(['C']))]

In [612]:
df_2004_2005['disease'] = 3
df_2004_2005.loc[df_2004_2005.MAIN_SICK.str[:1].isin(['C']),'disease'] = 1
df_2004_2005.loc[((df_2004_2005.MAIN_SICK.str[:3].isin(['E11'])) | (df_2004_2005.SUB_SICK.str[:3].isin(['E11']))), 'disease'] = 2
aa = df_2004_2005.sort_values(by=['RECU_FR_DT','disease'], ascending=True).groupby("PERSON_ID").head(1)


In [615]:
len(df_2004_2005.PERSON_ID.unique())

67766

In [618]:
patient_2004_2005 = set(aa[aa.disease != 1].PERSON_ID) - exclude_2002_2003

In [619]:
len(patient_2004_2005)

25349

In [684]:
target_df = df_2004_2005[df_2004_2005.PERSON_ID.isin(list(patient_2004_2005))]

In [685]:
target_df = target_df.sort_values(by=['PERSON_ID','RECU_FR_DT'],ascending=True)
target_df

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease
547,10057177.0,E11,E784,20050912,2
528,10057177.0,E148,NaN,20050914,3
546,10057177.0,E11,D649,20050915,2
527,10057177.0,E148,NaN,20051014,3
548,10057177.0,E11,D649,20051014,2
...,...,...,...,...,...
6267239,93788768.0,E145,N40,20040407,3
6267237,93788768.0,J189,E145,20040510,3
6267243,93829831.0,I10,E115,20041221,2
6267244,93852934.0,E145,I10,20040302,3


In [687]:
target_df = target_df.reset_index(drop=True)

In [697]:
pd.DataFrame(target_df.loc[target_df[target_df.disease==2].index[0],]).T

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease
0,1.00572e+07,E11,E784,20050912,2


In [703]:
kk = pd.DataFrame()
for i in tqdm(target_df.PERSON_ID.unique()):
    sung1 = target_df[target_df.PERSON_ID == i]
    if sum(sung1.disease ==1)>0:
        don1 = sung1.loc[:sung1[sung1.disease == 1].index[0]-1,]
        if sum(don1.disease ==2) >0:
            kk = pd.concat([kk,pd.DataFrame(don1.loc[don1[don1.disease==2].index[0],]).T])
        else:
            kk = pd.concat([kk,pd.DataFrame(don1.loc[don1[don1.disease==3].index[0],]).T])
    else:
        if sum(sung1.disease ==2) >0:
            kk = pd.concat([kk,pd.DataFrame(sung1.loc[sung1[sung1.disease==2].index[0],]).T])
        else:
            kk = pd.concat([kk,pd.DataFrame(sung1.loc[sung1[sung1.disease==3].index[0],]).T]) 

In [707]:
first_patient = kk.copy()

In [708]:
tmp_2004 = pd.read_sas('D:\\코호트자료\\표본코호트\\01.자격\\nhid_jk_2004.sas7bdat',format='sas7bdat',encoding='iso-8859-1')
tmp_2005 = pd.read_sas('D:\\코호트자료\\표본코호트\\01.자격\\nhid_jk_2005.sas7bdat',format='sas7bdat',encoding='iso-8859-1')

In [709]:
tmp_dual = pd.concat([tmp_2004,tmp_2005])

In [711]:
len(tmp_dual.PERSON_ID.unique())

1038753

In [726]:
first_patient

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease
0,1.00572e+07,E11,E784,20050912,2
5,1.01042e+07,I10,E11,20050729,2
6,1.01075e+07,E12,I10,20040726,3
24,1.01189e+07,E11,M818,20050812,2
30,1.01202e+07,E149,K529,20050323,3
...,...,...,...,...,...
99909,9.36338e+07,E119,NaN,20051101,2
99912,9.37888e+07,E145,N40,20040302,3
99915,9.38298e+07,I10,E115,20041221,2
99916,9.38529e+07,E145,I10,20040302,3


In [728]:
first_patient['RECU_FR_DT'] = first_patient.RECU_FR_DT.astype('str')

In [732]:
first_patient['STND_Y'] = first_patient.RECU_FR_DT.str[:4]

In [737]:
first_patient1 = first_patient.merge(tmp_dual,on=['PERSON_ID','STND_Y'])

In [739]:
calc_df = first_patient1.copy()

In [740]:
calc_df

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease,STND_Y,SEX,AGE_GROUP,DTH_YM,DTH_CODE1,DTH_CODE2,SIDO,SGG,IPSN_TYPE_CD,CTRB_PT_TYPE_CD,DFAB_GRD_CD,DFAB_PTN_CD,DFAB_REG_YM
0,1.00572e+07,E11,E784,20050912,2,2005,2,16,NaN,NaN,NaN,26,26230,1,1,0,0,NaN
1,1.01042e+07,I10,E11,20050729,2,2005,1,15,NaN,NaN,NaN,11,11680,6,7,0,0,NaN
2,1.01075e+07,E12,I10,20040726,3,2004,2,15,NaN,NaN,NaN,41,41460,6,7,0,0,NaN
3,1.01189e+07,E11,M818,20050812,2,2005,2,15,NaN,NaN,NaN,41,41820,6,5,0,0,NaN
4,1.01202e+07,E149,K529,20050323,3,2005,1,15,NaN,NaN,NaN,27,27140,6,9,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25344,9.36338e+07,E119,NaN,20051101,2,2005,1,12,NaN,NaN,NaN,41,41360,6,7,0,0,NaN
25345,9.37888e+07,E145,N40,20040302,3,2004,1,16,NaN,NaN,NaN,45,45800,7,0,0,0,200001
25346,9.38298e+07,I10,E115,20041221,2,2004,2,15,NaN,NaN,NaN,11,11500,7,0,0,0,NaN
25347,9.38529e+07,E145,I10,20040302,3,2004,1,14,NaN,NaN,NaN,28,28170,7,0,0,0,198607


In [743]:
calc_df = calc_df.iloc[:,:8]

In [746]:
calc_df.AGE_GROUP = calc_df.AGE_GROUP.astype(int)

In [747]:
calc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25349 entries, 0 to 25348
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PERSON_ID   25349 non-null  object
 1   MAIN_SICK   25349 non-null  object
 2   SUB_SICK    20076 non-null  object
 3   RECU_FR_DT  25349 non-null  object
 4   disease     25349 non-null  object
 5   STND_Y      25349 non-null  object
 6   SEX         25349 non-null  object
 7   AGE_GROUP   25349 non-null  int32 
dtypes: int32(1), object(7)
memory usage: 1.6+ MB


In [750]:
calc_df = calc_df[calc_df.AGE_GROUP >=9]

In [755]:
calc_df.groupby(['AGE_GROUP','disease']).count()

PERSON_ID  MAIN_SICK  SUB_SICK  RECU_FR_DT  STND_Y   SEX
AGE_GROUP disease                                                          
9         2             1549       1549      1177        1549    1549  1549
          3              607        607       461         607     607   607
10        2             2279       2279      1781        2279    2279  2279
          3              891        891       707         891     891   891
11        2             2304       2304      1818        2304    2304  2304
          3              967        967       759         967     967   967
12        2             2275       2275      1812        2275    2275  2275
          3              826        826       673         826     826   826
13        2             2297       2297      1825        2297    2297  2297
          3              794        794       639         794     794   794
14        2             1923       1923      1594        1923    1923  1923
          3              785        785       642         785     785   785
15        2             1402       1402      1174        1402    1402  1402
          3              533        533       439         533     533   533
16        2              741        741       615         741     741   741
          3              292        292       237         292     292   292
17        2              356        356       305         356     356   356
          3              161        161       140         161     161   161
18        2              144        144       128         144     144   144
          3               60         60        56          60      60    60

In [760]:
a = 607
b = 891
c = 967
d = 826
e = 794
a/(a+b+c+d+e)*100, b/(a+b+c+d+e)*100, c/(a+b+c+d+e)*100, d/(a+b+c+d+e)*100, e/(a+b+c+d+e)*100

(14.859241126070991,
 21.811505507955935,
 23.671970624235005,
 20.220318237454098,
 19.436964504283967)

In [777]:
data_set = pd.DataFrame()
for i in tqdm(['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013']):
    test = pd.read_csv('patient_'+i+'.csv',index_col=0)
    test['testing'] = 2
    test.loc[(test.MAIN_SICK.str[0] == 'C'),'testing'] = 1
    test2 = test[test.testing==1]
    test3 = test2.sort_values(by=['RECU_FR_DT'], ascending=True).groupby("PERSON_ID").head(1)
    data_set = pd.concat([data_set,test3])

In [779]:
data_set.testing.unique()

array([1], dtype=int64)

In [780]:
data_set2 = data_set.sort_values(by=['RECU_FR_DT'],ascending=True).groupby('PERSON_ID').head(1)
data_set2

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,testing
37658,12251617.0,C34,C349,20040101,1
488584,18481588.0,C34,R50,20040101,1
917375,24545235.0,C56,B373,20040101,1
1080917,26845611.0,C509,NaN,20040101,1
1381320,30590999.0,C679,D630,20040101,1
...,...,...,...,...,...
934709,16429727.0,C569,D62,20131231,1
754795,15233890.0,C5099,M8190,20131231,1
12343648,91488852.0,C3499,J984,20131231,1
5769843,48365415.0,C73,M79198,20131231,1


In [768]:
calc_df

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease,STND_Y,SEX,AGE_GROUP
0,1.00572e+07,E11,E784,20050912,2,2005,2,16
1,1.01042e+07,I10,E11,20050729,2,2005,1,15
2,1.01075e+07,E12,I10,20040726,3,2004,2,15
3,1.01189e+07,E11,M818,20050812,2,2005,2,15
4,1.01202e+07,E149,K529,20050323,3,2005,1,15
...,...,...,...,...,...,...,...,...
25344,9.36338e+07,E119,NaN,20051101,2,2005,1,12
25345,9.37888e+07,E145,N40,20040302,3,2004,1,16
25346,9.38298e+07,I10,E115,20041221,2,2004,2,15
25347,9.38529e+07,E145,I10,20040302,3,2004,1,14


In [769]:
cancer_start = data_set2[data_set2.PERSON_ID.isin(calc_df.PERSON_ID.unique())]

In [772]:
cancer_start2 = cancer_start.merge(calc_df, on=['PERSON_ID'])
cancer_start2

,PERSON_ID,MAIN_SICK_x,SUB_SICK_x,RECU_FR_DT_x,testing,MAIN_SICK_y,SUB_SICK_y,RECU_FR_DT_y,disease,STND_Y,SEX,AGE_GROUP
0,4.27432e+07,C220,NaN,20040116,1,E14,K739,20040114,3,2004,1,14
1,2.81509e+07,C34,R599,20040203,1,J41,E11,20040202,2,2004,2,15
2,1.24651e+07,C220,K746,20040205,1,K746,E118,20040114,2,2004,1,13
3,1.26211e+07,C16,C77,20040207,1,K566,E11,20040120,2,2004,1,12
4,1.27234e+07,C80,C786,20040217,1,E119,NaN,20040212,2,2004,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...
2300,8.01914e+07,C240,K293,20131224,1,B169,E119,20040616,2,2004,2,15
2301,5.46991e+07,C252,C797,20131226,1,E11,I11,20040602,2,2004,2,17
2302,6.23842e+07,C61,N400,20131230,1,I10,E119,20040303,2,2004,1,14
2303,7.97888e+07,C240,I109,20131230,1,E11,H10,20050824,2,2005,1,12


In [773]:
cancer_start2.groupby(['disease']).count()

,PERSON_ID,MAIN_SICK_x,SUB_SICK_x,RECU_FR_DT_x,testing,MAIN_SICK_y,SUB_SICK_y,RECU_FR_DT_y,STND_Y,SEX,AGE_GROUP
disease,,,,,,,,,,,
2,1637,1637,1090,1637,1637,1637,1352,1637,1637,1637,1637
3,668,668,459,668,668,668,536,668,668,668,668


In [776]:
a = 5248
b = 668
a/(a+b)*100, b/(a+b)*100

(88.70858688302907, 11.291413116970926)

In [783]:
cancer_start2.RECU_FR_DT_x = cancer_start2.RECU_FR_DT_x.astype('str')
cancer_start2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2305 entries, 0 to 2304
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PERSON_ID     2305 non-null   object
 1   MAIN_SICK_x   2305 non-null   object
 2   SUB_SICK_x    1549 non-null   object
 3   RECU_FR_DT_x  2305 non-null   object
 4   testing       2305 non-null   int64 
 5   MAIN_SICK_y   2305 non-null   object
 6   SUB_SICK_y    1888 non-null   object
 7   RECU_FR_DT_y  2305 non-null   object
 8   disease       2305 non-null   object
 9   STND_Y        2305 non-null   object
 10  SEX           2305 non-null   object
 11  AGE_GROUP     2305 non-null   int32 
dtypes: int32(1), int64(1), object(10)
memory usage: 225.1+ KB


In [785]:
cancer_start2.RECU_FR_DT_x = pd.to_datetime(cancer_start2.RECU_FR_DT_x)
cancer_start2.RECU_FR_DT_y = pd.to_datetime(cancer_start2.RECU_FR_DT_y)
cancer_start2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2305 entries, 0 to 2304
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   PERSON_ID     2305 non-null   object        
 1   MAIN_SICK_x   2305 non-null   object        
 2   SUB_SICK_x    1549 non-null   object        
 3   RECU_FR_DT_x  2305 non-null   datetime64[ns]
 4   testing       2305 non-null   int64         
 5   MAIN_SICK_y   2305 non-null   object        
 6   SUB_SICK_y    1888 non-null   object        
 7   RECU_FR_DT_y  2305 non-null   datetime64[ns]
 8   disease       2305 non-null   object        
 9   STND_Y        2305 non-null   object        
 10  SEX           2305 non-null   object        
 11  AGE_GROUP     2305 non-null   int32         
dtypes: datetime64[ns](2), int32(1), int64(1), object(8)
memory usage: 225.1+ KB


In [787]:
cancer_start2

,PERSON_ID,MAIN_SICK_x,SUB_SICK_x,RECU_FR_DT_x,testing,MAIN_SICK_y,SUB_SICK_y,RECU_FR_DT_y,disease,STND_Y,SEX,AGE_GROUP
0,4.27432e+07,C220,NaN,2004-01-16,1,E14,K739,2004-01-14,3,2004,1,14
1,2.81509e+07,C34,R599,2004-02-03,1,J41,E11,2004-02-02,2,2004,2,15
2,1.24651e+07,C220,K746,2004-02-05,1,K746,E118,2004-01-14,2,2004,1,13
3,1.26211e+07,C16,C77,2004-02-07,1,K566,E11,2004-01-20,2,2004,1,12
4,1.27234e+07,C80,C786,2004-02-17,1,E119,NaN,2004-02-12,2,2004,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...
2300,8.01914e+07,C240,K293,2013-12-24,1,B169,E119,2004-06-16,2,2004,2,15
2301,5.46991e+07,C252,C797,2013-12-26,1,E11,I11,2004-06-02,2,2004,2,17
2302,6.23842e+07,C61,N400,2013-12-30,1,I10,E119,2004-03-03,2,2004,1,14
2303,7.97888e+07,C240,I109,2013-12-30,1,E11,H10,2005-08-24,2,2005,1,12


In [790]:
cancer_start2['times'] = cancer_start2.RECU_FR_DT_x - cancer_start2.RECU_FR_DT_y
cancer_start2

,PERSON_ID,MAIN_SICK_x,SUB_SICK_x,RECU_FR_DT_x,testing,MAIN_SICK_y,SUB_SICK_y,RECU_FR_DT_y,disease,STND_Y,SEX,AGE_GROUP,times
0,4.27432e+07,C220,NaN,2004-01-16,1,E14,K739,2004-01-14,3,2004,1,14,2 days
1,2.81509e+07,C34,R599,2004-02-03,1,J41,E11,2004-02-02,2,2004,2,15,1 days
2,1.24651e+07,C220,K746,2004-02-05,1,K746,E118,2004-01-14,2,2004,1,13,22 days
3,1.26211e+07,C16,C77,2004-02-07,1,K566,E11,2004-01-20,2,2004,1,12,18 days
4,1.27234e+07,C80,C786,2004-02-17,1,E119,NaN,2004-02-12,2,2004,2,15,5 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,8.01914e+07,C240,K293,2013-12-24,1,B169,E119,2004-06-16,2,2004,2,15,3478 days
2301,5.46991e+07,C252,C797,2013-12-26,1,E11,I11,2004-06-02,2,2004,2,17,3494 days
2302,6.23842e+07,C61,N400,2013-12-30,1,I10,E119,2004-03-03,2,2004,1,14,3589 days
2303,7.97888e+07,C240,I109,2013-12-30,1,E11,H10,2005-08-24,2,2005,1,12,3050 days


In [820]:
cancer_start2['new'] = cancer_start2['times'].values.astype(np.int64)

means = cancer_start2.groupby('disease').mean()
means['new'] = pd.to_timedelta(means['new'])

## 교차검정

In [837]:
a = pd.DataFrame(np.array([13633,5248,1637,668]).reshape(2,2))

In [826]:
import scipy.stats as stats

In [845]:
calc_df['cancer'] = 0
calc_df.loc[calc_df.PERSON_ID.isin(cancer_start2.PERSON_ID.unique()),'cancer'] = 1

In [846]:
calc_df

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease,STND_Y,SEX,AGE_GROUP,cancer
0,1.00572e+07,E11,E784,20050912,2,2005,2,16,0
1,1.01042e+07,I10,E11,20050729,2,2005,1,15,1
2,1.01075e+07,E12,I10,20040726,3,2004,2,15,1
3,1.01189e+07,E11,M818,20050812,2,2005,2,15,0
4,1.01202e+07,E149,K529,20050323,3,2005,1,15,0
...,...,...,...,...,...,...,...,...,...
25344,9.36338e+07,E119,NaN,20051101,2,2005,1,12,0
25345,9.37888e+07,E145,N40,20040302,3,2004,1,16,1
25346,9.38298e+07,I10,E115,20041221,2,2004,2,15,0
25347,9.38529e+07,E145,I10,20040302,3,2004,1,14,0


In [848]:
calc_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21186 entries, 0 to 25348
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PERSON_ID   21186 non-null  object
 1   MAIN_SICK   21186 non-null  object
 2   SUB_SICK    16982 non-null  object
 3   RECU_FR_DT  21186 non-null  object
 4   disease     21186 non-null  object
 5   STND_Y      21186 non-null  object
 6   SEX         21186 non-null  object
 7   AGE_GROUP   21186 non-null  int32 
 8   cancer      21186 non-null  int64 
dtypes: int32(1), int64(1), object(7)
memory usage: 1.5+ MB


In [839]:
stats.chi2_contingency(observed = a)

(1.375756156935266,
 0.24082535295483248,
 1,
 array([[13608.65052393,  5272.34947607],
        [ 1661.34947607,   643.65052393]]))

In [815]:
cancer_start2.times_str.str.split('d')

0          [2 , ays]
1          [1 , ays]
2         [22 , ays]
3         [18 , ays]
4          [5 , ays]
            ...     
2300    [3478 , ays]
2301    [3494 , ays]
2302    [3589 , ays]
2303    [3050 , ays]
2304    [3647 , ays]
Name: times_str, Length: 2305, dtype: object

In [892]:
calc_df

,PERSON_ID,MAIN_SICK,SUB_SICK,RECU_FR_DT,disease,STND_Y,SEX,AGE_GROUP,cancer
0,1.00572e+07,E11,E784,20050912,2,2005,2,16,0
1,1.01042e+07,I10,E11,20050729,2,2005,1,15,1
2,1.01075e+07,E12,I10,20040726,3,2004,2,15,1
3,1.01189e+07,E11,M818,20050812,2,2005,2,15,0
4,1.01202e+07,E149,K529,20050323,3,2005,1,15,0
...,...,...,...,...,...,...,...,...,...
25344,9.36338e+07,E119,NaN,20051101,2,2005,1,12,0
25345,9.37888e+07,E145,N40,20040302,3,2004,1,16,1
25346,9.38298e+07,I10,E115,20041221,2,2004,2,15,0
25347,9.38529e+07,E145,I10,20040302,3,2004,1,14,0


In [893]:
odds = calc_df[['SEX','AGE_GROUP','cancer','disease']]

In [894]:
odds.loc[odds.AGE_GROUP==10,'AGE_GROUP'] = 9
odds.loc[odds.AGE_GROUP==12,'AGE_GROUP'] = 11
odds.loc[odds.AGE_GROUP==14,'AGE_GROUP'] = 13
odds.loc[odds.AGE_GROUP==16,'AGE_GROUP'] = 15
odds.loc[odds.AGE_GROUP==18,'AGE_GROUP'] = 17

In [895]:
odds.AGE_GROUP.unique()

array([15, 13, 11,  9, 17])

In [897]:
odds.AGE_GROUP = odds.AGE_GROUP.astype('object')

In [900]:
odds.cancer = odds.cancer.astype('object')

In [906]:
odds.disease = odds.disease.astype('int')

In [910]:
odds.loc[odds.disease == 2, 'disease'] = 0
odds.loc[odds.disease == 3, 'disease'] = 1


In [914]:
import statsmodels.api as sm

logis = sm.Logit.from_formula('disease ~ AGE_GROUP + SEX + cancer', odds).fit()

print(logis.summary())

Optimization terminated successfully.
         Current function value: 0.591246
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                disease   No. Observations:                21186
Model:                          Logit   Df Residuals:                    21179
Method:                           MLE   Df Model:                            6
Date:                Sat, 27 Nov 2021   Pseudo R-squ.:                0.001670
Time:                        13:58:01   Log-Likelihood:                -12526.
converged:                       True   LL-Null:                       -12547.
Covariance Type:            nonrobust   LLR p-value:                 1.912e-07
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.0223      0.034    -30.445      0.000      -1.088      -0.957
AGE_GROUP[T.11

In [913]:
np.exp(logis.params)

Intercept          0.359754
AGE_GROUP[T.11]    0.987024
AGE_GROUP[T.13]    0.926800
AGE_GROUP[T.15]    0.939806
AGE_GROUP[T.17]    1.076281
SEX[T.2]           1.204695
cancer[T.1]        1.090738
dtype: float64